# <font style="color:rgb(50, 120, 229);"> Probar un modelo de clasificación en tiempo real </font>

En este cuadermo, vamos a probar un modelo de clasificación en tiempo real.

Para ello, vamos a utilizar un modelo previamente entrenado y guardado en un archivo `.keras` o `.h5`.

Nos apoyaremos en la librería `keras` para cargar el modelo y hacer predicciones en tiempo real y de la librería `opencv` para capturar el video de una cámara web.

In [16]:
class_names = ['Fresh apple',
               'Fresh banana',
               'Fresh orange',
               'Rotten apple',
               'Rotten banana',
               'Rotten orange']

In [19]:
def predict_image(original_image, model):
    #Copiamos la imagen para no modificar la original
    image = original_image.copy()
    #Reescalamos la imagen al tamaño que espera el modelo (224x224)
    image = cv2.resize(image, (224, 224))

    #Convertimos la imagen a un tensor
    image  = image.reshape(1, 224, 224, 3)

    # Si el modelo incluye una capa de normalización, no es necesario normalizar la imagen
    #image = image / 255.0
    prediction = model.predict(image, verbose=0)

    #Obtenemos el índice de la clase con mayor probabilidad
    class_index = np.argmax(prediction)
    class_name = class_names[class_index]

    return class_name

In [20]:
import cv2
import numpy as np
from keras.models import load_model

# Cargamos el modelo de red neuronal que hemos entrenado previamente
model = load_model('./fruits.keras')

# Creamos un objeto VideoCapture para capturar la imagen de la cámara
# El parámetro 0 indica que se usará la cámara por defecto
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: No se puede abrir la cámara.")
    exit()


while True:
    # Capturamos un frame de la cámara
    ret, frame = cap.read()

    # Si no se ha podido capturar el frame, salimos del bucle
    if not ret:
        break

    # Comprobamos si se ha pulsado una tecla
    key = cv2.waitKey(1)

    # Si se ha pulsado la tecla 'q', salimos del bucle
    if key == ord('q'):
        break

    label = predict_image(frame, model)

    # Mostramos el texto con el nombre de la clase predicha
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('frame', frame)

cap.release()
cv2.destroyAllWindows()